# Aplicação 3: Marketing de Produto Bancário

## Dataset e Transformações

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from feature_engine.discretisation import EqualFrequencyDiscretiser, EqualWidthDiscretiser
from feature_engine.selection import DropConstantFeatures, SelectByInformationValue, SelectBySingleFeaturePerformance

In [2]:
marketing = pd.read_excel("bank_marketing.xlsx", skiprows=1).iloc[:,1:]

In [3]:
marketing.head()

,Cliente_Comprou_o_Titulo?,Idade,Profissão,Estado_Civil,Formação,Cliente_Devedor?,Saldo_Conta_Corrente,Tem_Hipoteca?,Tem_Emprestimo?,Qte_de_Ligações_Feitas
0,Não,33,Operário,Casado,Ensino Médio,Não,263,Sim,Não,3
1,Não,58,Aposentado,Casado,Ensino Médio,Não,19,Não,Sim,3
2,Sim,83,Aposentado,Casado,Fundamental,Não,425,Não,Não,1
3,Sim,32,Administrador,Solteiro,Ensino Superior,Não,10281,Não,Não,1
4,Sim,31,Administrador,Solteiro,Ensino Médio,Não,360,Sim,Não,1


## Análise

- a) Para o marketing em redes sociais, a plataforma permite a seleção do público baseado em até 2 variáveis dentre as seguintes: idade, profissão, estado civil e escolaridade. Quais variáveis você escolheria com o objetivo de separar as pessoas que compram ou não o produto. (Dica: utilize o IV  para as variáveis acima e avalie os resultados)

In [28]:
df_foco_variaveis = marketing.iloc[:, 0:5]

In [29]:
df_foco_variaveis.head(2)

,Cliente_Comprou_o_Titulo?,Idade,Profissão,Estado_Civil,Formação
0,Não,33,Operário,Casado,Ensino Médio
1,Não,58,Aposentado,Casado,Ensino Médio


In [30]:
efd = EqualFrequencyDiscretiser(q = 5)
efd.fit(df_foco_variaveis[["Idade"]])
df_foco_variaveis["faixas_etarias"] = efd.transform(df_foco_variaveis[["Idade"]])

In [31]:
df_foco_variaveis.groupby("faixas_etarias").agg({"Idade": ["min", "max", "size"]}).reset_index()

faixas_etarias Idade          
                   min max  size
0              0    18  31  2222
1              1    32  36  2008
2              2    37  42  1674
3              3    43  52  1946
4              4    53  95  1834

In [32]:
df_foco_variaveis["faixas_etarias"] = (df_foco_variaveis["faixas_etarias"]
                                       .map({0: "18 a 31",
                                             1: "32 a 36",
                                             2: "37 a 42",
                                             3: "48 a 52",
                                             4: "53 a 95"}))

In [33]:
df_foco_variaveis = df_foco_variaveis.drop(columns={"Idade"})

In [34]:
df_foco_variaveis.head()

,Cliente_Comprou_o_Titulo?,Profissão,Estado_Civil,Formação,faixas_etarias
0,Não,Operário,Casado,Ensino Médio,32 a 36
1,Não,Aposentado,Casado,Ensino Médio,53 a 95
2,Sim,Aposentado,Casado,Fundamental,53 a 95
3,Sim,Administrador,Solteiro,Ensino Superior,32 a 36
4,Sim,Administrador,Solteiro,Ensino Médio,18 a 31


In [35]:
dcf = DropConstantFeatures()
dcf.fit(df_foco_variaveis)
dcf.transform(df_foco_variaveis)

,Cliente_Comprou_o_Titulo?,Profissão,Estado_Civil,Formação,faixas_etarias
0,Não,Operário,Casado,Ensino Médio,32 a 36
1,Não,Aposentado,Casado,Ensino Médio,53 a 95
2,Sim,Aposentado,Casado,Fundamental,53 a 95
3,Sim,Administrador,Solteiro,Ensino Superior,32 a 36
4,Sim,Administrador,Solteiro,Ensino Médio,18 a 31
...,...,...,...,...,...
9679,Não,Desempregado,Solteiro,Ensino Superior,18 a 31
9680,Sim,Aposentado,Casado,Ensino Médio,53 a 95
9681,Sim,Administrador,Solteiro,Ensino Superior,37 a 42
9682,Sim,Técnico,Solteiro,Ensino Médio,37 a 42


In [79]:
selecIV = SelectByInformationValue(threshold=0.1)
selecIV.fit(df_foco_variaveis.drop(columns={"Cliente_Comprou_o_Titulo?"}), df_foco_variaveis["Cliente_Comprou_o_Titulo?"])

SelectByInformationValue(threshold=0.1)

In [80]:
selecIV.information_values_

{'Profissão': 0.14961345004241128,
 'Estado_Civil': 0.034193461764569305,
 'Formação': 0.043708047182739504,
 'faixas_etarias': 0.06291495756607425}

In [81]:
selecIV.features_to_drop_

['Estado_Civil', 'Formação', 'faixas_etarias']

- b) Sobre a linha de comunicação, foi sugerido direcionar ao público com idade entre 30-40, pois, dos 4.778 clientes que compraram o produto, essa foi a faixa com maior quantidade. Você concorda com essa análise? Qual seria a sua sugestão?

In [13]:
marketing["is_30_40"] = np.where((marketing["Idade"] >= 30) & (marketing["Idade"] <= 40), True, False)

In [22]:
(marketing.loc[marketing["Cliente_Comprou_o_Titulo?"] == "Sim"]
 .groupby("Cliente_Comprou_o_Titulo?")[["is_30_40"]]
 .value_counts()
 .sort_values(ascending=False)
 .reset_index())

,Cliente_Comprou_o_Titulo?,is_30_40,count
0,Sim,False,2943
1,Sim,True,1835


In [19]:
(df_foco_variaveis.loc[df_foco_variaveis["Cliente_Comprou_o_Titulo?"] == "Sim"]
    .groupby("Cliente_Comprou_o_Titulo?")[["faixas_etarias"]]
    .value_counts()
    .sort_values(ascending = False).reset_index())

,Cliente_Comprou_o_Titulo?,faixas_etarias,count
0,Sim,48 a 52,82
1,Sim,53 a 95,71
2,Sim,37 a 42,69
3,Sim,18 a 31,68
4,Sim,32 a 36,68
